<a href="https://colab.research.google.com/github/aravinddddddd/Core-Banking-Automation-Finacle-API-Integration-System/blob/main/Core-Banking-Automation-Finacle-API-Integration-System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import random
import os

os.makedirs('data', exist_ok=True)
with open('data/customers.csv','w',newline='') as f:
    w = csv.writer(f)
    w.writerow(['customer_id','name','account_type','balance','status'])
    for i in range(1,50001):
        name = f"Cust{i}"
        acct = 'Savings' if i%2==0 else 'Current'
        balance = round(random.uniform(0,200000),2)
        status = 'Active' if random.random() < 0.9 else 'Inactive'
        w.writerow([i, name, acct, balance, status])
print("Created 50,000 records.")

Created 50,000 records.


In [2]:
# scripts/data_loader.py
import pandas as pd
import sqlite3
import os

DATA_FILE = os.path.join('data', 'customers.csv')
DB_FILE = os.path.join('db', 'bank.db')
SCHEMA = '''
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT,
    account_type TEXT,
    balance REAL,
    status TEXT
);
'''

# read CSV
df = pd.read_csv(DATA_FILE)

# trim whitespace on column names
df.columns = [c.strip() for c in df.columns]

# required fields
required = ['customer_id', 'name', 'account_type', 'balance', 'status']

# count rows that have ALL required fields non-empty/non-null
# For safety, treat empty string or NaN as missing
complete_mask = df[required].applymap(lambda x: not (pd.isna(x) or (isinstance(x, str) and x.strip() == ''))).all(axis=1)
complete_rows = df[complete_mask]
mapping_accuracy = len(complete_rows) / len(df) if len(df) > 0 else 0

# create db folder and write
os.makedirs('db', exist_ok=True)
conn = sqlite3.connect(DB_FILE)
conn.executescript(SCHEMA)
# store only complete rows to simulate onboarding
complete_rows.to_sql('customers', conn, if_exists='replace', index=False)
conn.commit()
conn.close()

print(f"Total rows in CSV: {len(df)}")
print(f"Complete rows loaded into DB: {len(complete_rows)}")
print(f"Data mapping accuracy: {mapping_accuracy * 100:.2f}%")
print(f"Database written to {DB_FILE}")

/tmp/ipython-input-188330634.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  complete_mask = df[required].applymap(lambda x: not (pd.isna(x) or (isinstance(x, str) and x.strip() == ''))).all(axis=1)


Total rows in CSV: 50000
Complete rows loaded into DB: 50000
Data mapping accuracy: 100.00%
Database written to db/bank.db


In [3]:
import pandas as pd
import sqlite3
import os

# paths
DATA_FILE = 'data/customers.csv'
DB_FILE = 'bank.db'  # simpler in colab, saves in current folder

# make sure file exists
if not os.path.exists(DATA_FILE):
    raise FileNotFoundError("customers.csv not found. Please run generator again!")

# read CSV
df = pd.read_csv(DATA_FILE)

# required columns
required = ['customer_id', 'name', 'account_type', 'balance', 'status']

# check completeness (98% good)
def is_complete(row):
    for col in required:
        val = row[col]
        if pd.isna(val) or (isinstance(val, str) and val.strip() == ''):
            return False
    return True

complete_rows = df[df.apply(is_complete, axis=1)]

# calculate mapping accuracy
mapping_accuracy = len(complete_rows) / len(df) * 100

# save to SQLite database
conn = sqlite3.connect(DB_FILE)
conn.execute('DROP TABLE IF EXISTS customers')
conn.execute('''
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT,
    account_type TEXT,
    balance REAL,
    status TEXT
)
''')

complete_rows.to_sql('customers', conn, if_exists='append', index=False)
conn.commit()
conn.close()

print("✅ Data loaded successfully!")
print(f"Total rows in CSV: {len(df)}")
print(f"Rows loaded to DB: {len(complete_rows)}")
print(f"Mapping accuracy: {mapping_accuracy:.2f}%")
print(f"Database file saved as: {DB_FILE}")

✅ Data loaded successfully!
Total rows in CSV: 50000
Rows loaded to DB: 50000
Mapping accuracy: 100.00%
Database file saved as: bank.db


In [4]:
import sqlite3

conn = sqlite3.connect('bank.db')
cur = conn.cursor()
cur.execute("SELECT COUNT(*) FROM customers")
rows = cur.fetchone()[0]
print("✅ Rows stored in database:", rows)

cur.execute("SELECT * FROM customers LIMIT 5")
print("\n🧾 Sample data:")
for row in cur.fetchall():
    print(row)

conn.close()

✅ Rows stored in database: 50000

🧾 Sample data:
(1, 'Cust1', 'Current', 165918.22, 'Active')
(2, 'Cust2', 'Savings', 27670.66, 'Inactive')
(3, 'Cust3', 'Current', 9212.44, 'Active')
(4, 'Cust4', 'Savings', 17399.88, 'Active')
(5, 'Cust5', 'Current', 102376.71, 'Active')


In [5]:
!pip install flask flask_ngrok

In [6]:
!pip install flask-pyngrok

  Preparing metadata (setup.py) ... done
  Created wheel for flask-pyngrok: filename=Flask_PyNgrok-1.0.3-py3-none-any.whl size=2164 sha256=740297eb24eeb06938e03f4e529b623cb7847467d0744045e7d2f668543f5e63
  Stored in directory: /root/.cache/pip/wheels/1f/6e/73/3f8ee856a722ea638966ec03f5920c05fb991ce4244652a0aa
Successfully built flask-pyngrok


In [7]:
from pyngrok import ngrok
ngrok.set_auth_token("34ma62Cm6goWk5PAQ256lFdiEei_64SxfanWRCJK7YeVAVGEM")

In [8]:
# ✅ Works in Colab using pyngrok
from flask import Flask, request, jsonify
from pyngrok import ngrok
import random, threading

app = Flask(__name__)

@app.route('/')
def home():
    return "✅ Finacle Mock API Running"

@app.route('/api/validateTransaction', methods=['POST'])
def validate_transaction():
    data = request.json or {}
    txn_id = data.get("txn_id", "unknown")
    amount = data.get("amount", 0)

    # Simulate 98% success rate
    if random.random() < 0.98:
        return jsonify({
            "status": "success",
            "txn_id": txn_id,
            "amount": amount,
            "message": "Transaction validated successfully"
        }), 200
    else:
        return jsonify({
            "status": "failed",
            "txn_id": txn_id,
            "reason": "Simulated transaction error"
        }), 400

# Open ngrok tunnel on port 5000
public_url = ngrok.connect(5000)
print("🌍 Public URL:", public_url)

# Run Flask in a background thread so Colab doesn't block
threading.Thread(target=lambda: app.run(port=5000)).start()

🌍 Public URL: NgrokTunnel: "https://carking-ashly-unspecializing.ngrok-free.dev" -> "http://localhost:5000"


In [9]:
import requests

url = "https://carking-ashly-unspecializing.ngrok-free.dev/api/validateTransaction"
payload = {"txn_id": "TXN12345", "amount": 5000}

r = requests.post(url, json=payload)
print("Response Code:", r.status_code)
print("Response JSON:", r.json())

INFO:werkzeug:127.0.0.1 - - [30/Oct/2025 15:44:41] "POST /api/validateTransaction HTTP/1.1" 200 -


Response Code: 200
Response JSON: {'amount': 5000, 'message': 'Transaction validated successfully', 'status': 'success', 'txn_id': 'TXN12345'}


In [10]:
import os, shutil

def deploy():
    print("🚀 Deploying Finacle API simulation...")
    os.makedirs("deployment_logs", exist_ok=True)
    with open("deployment_logs/deploy.log", "w") as f:
        f.write("Deployment successful\n")
    print("✅ Deployment completed successfully.")

deploy()

🚀 Deploying Finacle API simulation...
✅ Deployment completed successfully.


In [11]:
import os

def rollback():
    print("🔁 Rolling back to previous stable version...")
    if os.path.exists("deployment_logs/deploy.log"):
        os.remove("deployment_logs/deploy.log")
        print("✅ Rollback completed successfully.")
    else:
        print("⚠ No deployment found to rollback.")

rollback()

🔁 Rolling back to previous stable version...
✅ Rollback completed successfully.
